<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px" />

# Excel Madness Lab!

_Author:_ Tim Book

## Our Mission
We work for a large supermarket chain, with stores in 10 major cities that happen to coincide with General Assembly campuses. However, this company's idea of a "database" is just a bunch of Excel spreadsheets! In order to analyze our data, we're going to need to process the existing data into a form we can use. **Our end goal is to have one csv per city.**

## Cleanup Duty!
It is a hard truth that data scientists spend a large majority of their time cleaning data. Data never arrives on our desks in exactly the format in which we want it, and it's up to us to transform it to a workable format.

Being good cleaning, moving, and reshaping data is in itself a valuable and employable job skill. If you follow these directions exactly, we will walk through constructing an automated process for processing data from this supermarket chain.

# Part I: Processing

### Step 1: Imports and the `os` library
We're going to import three libraries: numpy, pandas, and `os`.

In [1]:
# Import libraries here.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

The `os` library is part of the Python standard library. It ships with Python. It is extremely useful for performing system commands from within Python. 

1. Create an `output` folder using `os.mkdir()`
**WARNING:** The `os.mkdir()` function will give you an error if you try to make a folder that already exists!

2. Use `os.listdir()` to create a variable called `files` that is the list of files in the `data` folder.



In [38]:
# Create an output folder.
os.mkdir('./output/')

# Create a files variable that contains all of our data files.
files = os.listdir('./data')

In [39]:
files

['Jan 23.xlsx',
 'Jan 19.xlsx',
 'Jan 15.xlsx',
 'Jan 7.xlsx',
 'Jan 6.xlsx',
 'Jan 14.xlsx',
 'Jan 18.xlsx',
 'Jan 22.xlsx',
 'Jan 29.xlsx',
 'Jan 13.xlsx',
 'Jan 1.xlsx',
 'Jan 25.xlsx',
 'Jan 24.xlsx',
 'Jan 12.xlsx',
 'Jan 28.xlsx',
 'Jan 3.xlsx',
 'Jan 11.xlsx',
 'Jan 27.xlsx',
 'Jan 31.xlsx',
 'Jan 30.xlsx',
 'Jan 26.xlsx',
 'Jan 10.xlsx',
 'Jan 2.xlsx',
 'Jan 9.xlsx',
 'Jan 21.xlsx',
 'Jan 5.xlsx',
 'Jan 17.xlsx',
 'Jan 16.xlsx',
 'Jan 4.xlsx',
 'Jan 20.xlsx',
 'Jan 8.xlsx']

### Step 2: Process one DataFrame
It looks like we have data for the month of January. 31 files of 10 sheets each! Luckily they are all in the same format. So let's read just one in and process that. It might be helpful to open one up in your spreadsheet viewer of choice first (Excel, Numbers, Sheets, etc.)

In [4]:
# Read in data from your city from January 1st.
df = pd.read_excel('./data/Jan 1.xlsx')
df

/Users/genebob/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,prodcode,price_eu,weight_kg,quantity
0,4159,2.048141,8.812961,138.0
1,4062,2.728485,9.331372,184.0
2,4404,2.182498,8.937798,491.0
3,4412,2.342694,7.067682,218.0
4,4131,1.802839,9.551503,391.0
...,...,...,...,...
124,4796,1.827676,7.922745,444.0
125,4225,2.463229,5.839095,258.0
126,4134,0.920152,9.225959,395.0
127,3302,1.510998,2.667668,168.0


### Step 2a: Convert to units common in the USA
For whatever reason, our data are stored in euros and kilograms. Create `price_usd` and `weight_lb` columns. There are 2.2 pounds per kilogram, and use 1.1 dollars per euro.

In [8]:
df['price_usd'] = df['price_eu'].map(lambda x: x * 1.1)
df['weight_lb'] = df['weight_kg'].map(lambda x: x * 2.2)

### Step 2b: Merge in product names
You'll notice we also have a `plu-codes.csv` file containing actual product names matched up against their price lookup (PLU) codes. Let's merge these product names onto our Jan 1 data.
* _Hint 1:_ What kind of merge is this? Right, left, inner, outer, etc.?
* _Hint 2:_ Pay special attention to column names!

In [10]:
plu = pd.read_csv("plu-codes.csv")
plu.head()

,product,plu_code
0,Apple (Fuji),4131
1,Apple (Gala),4134
2,Apricot,3302
3,Avocado,4225
4,Banana,4011


In [12]:
df.columns

Index(['prodcode', 'price_eu', 'weight_kg', 'quantity', 'price_usd',
       'weight_lb'],
      dtype='object')

In [16]:
df = pd.merge(left=df, right=plu, how='left', left_on='prodcode', right_on='plu_code')

### Step 2c: Drop unnecessary columns
We've created some extraneous columns. Drop the old price and weight columns, as well as any redundant columns.

In [20]:
df.drop(columns=['price_eu', 'weight_kg', 'plu_code'], inplace=True)

### Step 2d: Add the date
Simply create a new `date` column that is the date this data was collected. For example, if this is from `Jan 1.xlsx`, this column should be full of `Jan 1`.

In [21]:
df['Jan 1'] = 'Jan 1'

In [22]:
df.head()

,prodcode,quantity,price_usd,weight_lb,product,Jan 1
0,4159,138.0,2.252955,19.388515,Onion,Jan 1
1,4062,184.0,3.001334,20.529018,Cucumber,Jan 1
2,4404,491.0,2.400748,19.663156,Peach,Jan 1
3,4412,218.0,2.576964,15.548899,Pear,Jan 1
4,4131,391.0,1.983123,21.013306,Apple (Fuji),Jan 1


### Step 3: Write a function that conducts all of Step 2
This function should import a **filename and a city name** and return a fully processed DataFrame. That is, the function should:
1. Read in the data from the given file and city.
1. Create USD and pound columns.
1. Merge in product names.
1. Drop unnecessary columns.
1. Add a date column

In [45]:
def process_data(file, city):
    # read .xlsx
    df_date = pd.read_excel(file, sheet_name=city)
    # convert price & weight
    df_date['price_usd'] = df_date['price_eu'].map(lambda x: x * 1.1)
    df_date['weight_lb'] = df_date['weight_kg'].map(lambda x: x * 2.2)
    # merge product names
    df_date = pd.merge(left=df_date, right=pd.read_csv('plu-codes.csv').rename(columns={'plu_code': 'prodcode'}), how='left', on='prodcode')
    # drop columns
    df_date.drop(columns=['price_eu', 'weight_kg'], inplace=True)
    # Add a date column
    df_date['date'] = os.path.basename(file).split(sep='.')[:1][0]

    return df_date

Test your function out on a new file and city!

In [46]:
process_data('./data/Jan 2.xlsx', 'New York')

/Users/genebob/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,prodcode,quantity,price_usd,weight_lb,product,date
0,4650,122.0,2.424953,3.396210,Mushroom,Jan 2
1,4309,133.0,1.860956,14.122334,Lychee,Jan 2
2,4650,473.0,0.965937,8.689899,Mushroom,Jan 2
3,4062,484.0,2.303898,4.290737,Cucumber,Jan 2
4,4011,259.0,2.818418,2.217087,Banana,Jan 2
...,...,...,...,...,...,...
125,4078,468.0,3.248011,21.394509,Corn,Jan 2
126,4225,270.0,2.793992,11.926459,Avocado,Jan 2
127,4012,302.0,1.330683,10.048707,Orange,Jan 2
128,4012,413.0,1.790014,12.295220,Orange,Jan 2


### Step 4: Process all of January's data
For each spreadsheet, process the data and store the resulting DataFrame in one big list. **You only need to do this for your city!**

* _Hint 1:_ A listcomp would make this whole step one line of code!
* _Hint 2:_ You've already made that `files` variable to help you here.

In [48]:
list_all_data = [

    process_data('./data/' + _, 'New York') for _ in files
]

/Users/genebob/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


### Step 5: Concatenate all DataFrames from Step 4 into one large DataFrame
* _Hint:_ Is there a function in `pandas` that can do something like this for us? This is also just one line of code!

In [53]:
df_all = pd.concat(list_all_data)

### Step 6: Do this for all cities, write data
Here's the big one. For each city, process and the data as in steps 3-5, and then write the data to our `output` folder. Below is a dictionary of city name to desired output file name.

Before writing your DataFrame, do the following:
* Add a `city` column
* Reorder the columns into the following order:


| city | date | product | prodcode | quantity | weight_lb | price_usd |
|---|---|---|---|---|---|---|


In [55]:
city_dict = {
    "Atlanta": "atl.csv",
    "Austin": "atx.csv",
    "Boston": "bos.csv",
    "Chicago": "chi.csv",
    "Denver": "den.csv",
    "Los Angeles": "lax.csv",
    "New York": "nyc.csv",
    "San Francisco": "sf.csv",
    "Seattle": "sea.csv",
    "Washington, DC": "dc.csv"
}



In [60]:
for key, value in city_dict.items():
    # make data frame
    df = pd.concat([process_data('./data/' + _, key) for _ in files])
    # add city column
    df['city'] = key
    # re-order columns
    df = df[['city', 'date', 'product', 'prodcode', 'quantity', 'weight_lb', 'price_usd']]
    # export to .csv
    df.to_csv('./output/' + value, index=False)

/Users/genebob/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


# Part II: Checking our answers 
In steps very similar to the ones conducted above...
1. Loop through the files we just wrote to `output`, and read them in, collecting them all in one list.
1. Combine all of those DataFrames into one large DataFrame.
1. For each city, find the mean `quantity`, `weight_lb`, and `price_usd`.

If you've done everything correct, your answer should look exactly like this:

![](imgs/correct-output.png)

In [75]:
pd.concat([pd.read_csv(d) for d in [os.path.join('./output', _) for _ in os.listdir('./output')]]).groupby(by='city')['quantity', 'weight_lb', 'price_usd'].mean()

/var/folders/3y/cshc407d2zv7031zkk2k4r1r0000gn/T/ipykernel_70338/3178099035.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  pd.concat([pd.read_csv(d) for d in [os.path.join('./output', _) for _ in os.listdir('./output')]]).groupby(by='city')['quantity', 'weight_lb', 'price_usd'].mean()


,quantity,weight_lb,price_usd
city,,,
Atlanta,302.485306,11.929027,1.937980
Austin,301.048298,12.092775,1.937456
Boston,298.806536,12.063057,1.900973
Chicago,301.686186,12.101555,1.930026
Denver,301.012745,12.129850,1.932088
Los Angeles,301.531276,12.167527,1.918331
New York,299.482863,12.090294,1.912662
San Francisco,298.979895,12.237399,1.925830
Seattle,300.333694,11.925486,1.903315


# Part III (BONUS): Get this process production-ready!
_This part of the lab is optional, but very highly recommended, as the skills developed in this part are extremely common in industry._

For this step, we're going to take this whole process and put it into a production-ready Python script. **PRODUCTIONALIZED ETL (_"Extract, Transform, Load"_) CODE DOES NOT TAKE PLACE IN A JUPYTER NOTEBOOK!**

The instructions are simple: As conducted in this lab, read, transform, and export the data in our Excel files into .csv files. The code should be in a `.py` file and executable from the command line. Here are some hints and tips to guide you:

### Hints, tips, and tricks:
* A good place to start is with the code you've already written. In this notebook, you can click `File > Export Notebook as as > Executable Script` to export as a `.py` file. Most of this exercise then comes down to you cleaning this file. (There will be a lot to clean).
* `os.mkdir()` will throw an error if the folder you're trying to make already exists. Maybe you should check to see if it already exists. If it already exists, what should you do? (Remember that `.csv` can be overwritten with no problem.) The functions that can help you with this are all in the `os` library.
* Remember to follow all of the Python best practices we've already learned:
    - All import statements should go at the top of your script.
    - Comment your code. Comments shouldn't explain _what_ code does, but _why_ the code does this.
    - Keep your code DRY (don't repeat yourself) as opposed to WET (write everything twice). All constants should be variables that only need to be changed once. All code should be bottled into functions so you only need to fix it once.
* Make sure not to hardcode "Jan" anywhere. The point is that this code can be run throughout the lifetime of this supermarket's project, which is likely months or years. Keep your code so that if you get February data, you only need to change one tiny piece of the script (probably a file path).